In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
import datetime
from itertools import compress
import seaborn as sns; sns.set(rc = {'figure.figsize':(15,8)})
from ks_functions import preprocessing, add_time_values, get_bs_neighbors, compute_bs_neighbors_stat, add_neighbours_price, add_mmvb_index
from outer_features import find_number, add_extra

In [2]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
idx_mmvb = pd.read_csv('idx_mmvb.csv')

/Users/ksenia/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data = preprocessing(train, test)
data

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type,data_type
0,Пермь,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,...,5.530612,1964.118519,1960.959184,Пермский край,32.000000,S27289,2020-01-05,10,0,train
1,Шатура,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,...,3.527778,1952.321678,1957.222222,Московская область,280.000000,S17052,2020-01-05,10,0,train
2,Ярославль,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,...,7.222222,1968.150000,1973.370370,Ярославская область,297.400000,S16913,2020-01-05,110,0,train
3,Новокузнецк,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,...,9.000000,1992.716216,2014.000000,Кемеровская область,190.000000,S10148,2020-01-05,110,0,train
4,Москва,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,...,5.684211,1963.229167,1960.500000,Москва,60.200000,S1338,2020-01-05,10,0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2969,Красноярск,4.0,COL_455089,55.978180,92.891457,0,3,6,16,0,...,6.333333,1972.153846,1982.250000,Красноярский край,2149.546362,S2650,2020-12-27,100,1,test
2970,Томск,1.0,COL_455212,56.459183,84.979872,2,33,111,222,0,...,5.882353,1972.260870,1973.460000,Томская область,359.371061,S11114,2020-12-27,10,1,test
2971,Калуга,-1.0,COL_455261,54.523247,36.295168,2,25,54,99,0,...,4.909091,1966.390244,1966.904762,Калужская область,87.820453,S34005,2020-12-27,100,1,test
2972,Нижний Новгород,NaN,COL_455381,56.328236,43.990039,13,70,114,158,0,...,4.529412,1938.331361,1946.529412,Нижегородская область,208.625735,S21753,2020-12-27,10,1,test


In [4]:
data = add_time_values(data)
data = add_neighbours_price(data)
data = add_mmvb_index(data, idx_mmvb)

In [5]:
data

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,price_type,data_type,month,year,day,nn_100m_price,nn_300m_price,nn_1000m_price,mmvb_lag1,mmvb_lag2
0,Пермь,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,...,0,train,1,2020,5,NaN,NaN,NaN,3045.0,2935.0
1,Шатура,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,...,0,train,1,2020,5,NaN,NaN,NaN,3045.0,2935.0
2,Ярославль,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,...,0,train,1,2020,5,NaN,NaN,NaN,3045.0,2935.0
3,Новокузнецк,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,...,0,train,1,2020,5,NaN,NaN,NaN,3045.0,2935.0
4,Москва,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,...,0,train,1,2020,5,NaN,NaN,182873.333333,3045.0,2935.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282761,Красноярск,4.0,COL_455089,55.978180,92.891457,0,3,6,16,0,...,1,test,12,2020,27,26209.197409,31896.484243,39460.658496,3107.0,2690.0
282762,Томск,1.0,COL_455212,56.459183,84.979872,2,33,111,222,0,...,1,test,12,2020,27,32673.511185,38980.026302,51095.353150,3107.0,2690.0
282763,Калуга,-1.0,COL_455261,54.523247,36.295168,2,25,54,99,0,...,1,test,12,2020,27,NaN,11443.296808,30283.505991,3107.0,2690.0
282764,Нижний Новгород,NaN,COL_455381,56.328236,43.990039,13,70,114,158,0,...,1,test,12,2020,27,63854.863923,59342.662610,70805.555874,3107.0,2690.0


In [6]:
data_2 = add_extra(train, test)

In [7]:
data_2

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,ipc_goods_year,ipc_build_month,ipc_build_year,miacr,ipc_base,ipc_chain,interest_rate,inc_per_capita,invest_residue,debts
0,Пермь,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,...,1.0287,1.0034,1.0277,6.15,1.029332,1.004,6.363636,30346.0,5.97009,0.584440
1,Шатура,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,...,1.0290,1.0010,1.0283,6.15,1.029332,1.004,6.363636,47064.0,64.79590,2.078555
2,Ярославль,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,...,1.0307,1.0032,1.0403,6.15,1.029332,1.004,6.363636,28114.0,2.28274,0.093407
3,Новокузнецк,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,...,1.0317,1.0008,1.0273,6.15,1.029332,1.004,6.363636,24167.0,11.40058,0.044784
4,Москва,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,...,1.0342,1.0018,1.0374,6.15,1.029332,1.004,6.363636,74608.0,712.80243,30.349952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282761,Красноярск,4.0,COL_455089,55.978180,92.891457,0,3,6,16,0,...,1.0545,1.0121,1.0416,4.06,1.071222,1.007,4.250000,31114.0,56.18257,0.456430
282762,Томск,1.0,COL_455212,56.459183,84.979872,2,33,111,222,0,...,1.0415,1.0084,1.0430,4.06,1.071222,1.007,4.250000,27862.0,0.47218,0.064707
282763,Калуга,-1.0,COL_455261,54.523247,36.295168,2,25,54,99,0,...,1.0560,1.0133,1.0389,4.06,1.071222,1.007,4.250000,30932.0,11.71588,0.074113
282764,Нижний Новгород,NaN,COL_455381,56.328236,43.990039,13,70,114,158,0,...,1.0619,1.0202,1.0434,4.06,1.071222,1.007,4.250000,32267.0,9.66878,0.123371


In [8]:
data = data.merge(data_2, how = 'left')

In [9]:
data

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,ipc_goods_year,ipc_build_month,ipc_build_year,miacr,ipc_base,ipc_chain,interest_rate,inc_per_capita,invest_residue,debts
0,Пермь,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,...,1.0287,1.0034,1.0277,6.15,1.029332,1.004,6.363636,30346.0,5.97009,0.584440
1,Шатура,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,...,1.0290,1.0010,1.0283,6.15,1.029332,1.004,6.363636,47064.0,64.79590,2.078555
2,Ярославль,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,...,1.0307,1.0032,1.0403,6.15,1.029332,1.004,6.363636,28114.0,2.28274,0.093407
3,Новокузнецк,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,...,1.0317,1.0008,1.0273,6.15,1.029332,1.004,6.363636,24167.0,11.40058,0.044784
4,Москва,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,...,1.0342,1.0018,1.0374,6.15,1.029332,1.004,6.363636,74608.0,712.80243,30.349952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282761,Красноярск,4.0,COL_455089,55.978180,92.891457,0,3,6,16,0,...,1.0545,1.0121,1.0416,4.06,1.071222,1.007,4.250000,31114.0,56.18257,0.456430
282762,Томск,1.0,COL_455212,56.459183,84.979872,2,33,111,222,0,...,1.0415,1.0084,1.0430,4.06,1.071222,1.007,4.250000,27862.0,0.47218,0.064707
282763,Калуга,-1.0,COL_455261,54.523247,36.295168,2,25,54,99,0,...,1.0560,1.0133,1.0389,4.06,1.071222,1.007,4.250000,30932.0,11.71588,0.074113
282764,Нижний Новгород,NaN,COL_455381,56.328236,43.990039,13,70,114,158,0,...,1.0619,1.0202,1.0434,4.06,1.071222,1.007,4.250000,32267.0,9.66878,0.123371


In [10]:
data.columns[-35:]

Index(['region', 'total_square', 'street', 'date', 'realty_type', 'price_type',
       'data_type', 'month', 'year', 'day', 'nn_100m_price', 'nn_300m_price',
       'nn_1000m_price', 'mmvb_lag1', 'mmvb_lag2', 'number_of_supply1',
       'number_of_supply2', 'price_dynamic1', 'price_dynamic2',
       'mean_sqm_price1', 'mean_sqm_price2', 'exp_time', 'ipc_all_month',
       'ipc_all_year', 'ipc_goods_month', 'ipc_goods_year', 'ipc_build_month',
       'ipc_build_year', 'miacr', 'ipc_base', 'ipc_chain', 'interest_rate',
       'inc_per_capita', 'invest_residue', 'debts'],
      dtype='object')

In [39]:
data.to_csv('full_data.csv')